# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770852515273                   -0.52    9.0         
  2   -2.772142324932       -2.89       -1.32    1.0    104ms
  3   -2.772170371954       -4.55       -2.51    1.0    120ms
  4   -2.772170672585       -6.52       -3.33    1.0    107ms
  5   -2.772170721109       -7.31       -3.86    2.0    124ms
  6   -2.772170722967       -8.73       -5.42    1.0    131ms
  7   -2.772170723012      -10.35       -5.27    3.0    139ms
  8   -2.772170723015      -11.48       -6.31    1.0    122ms
  9   -2.772170723015      -13.46       -6.66    2.0    137ms
 10   -2.772170723015      -14.21       -7.14    1.0    159ms
 11   -2.772170723015      -14.18       -8.37    1.0    126ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770775076920                   -0.53    9.0         
  2   -2.7

1.7735577017380832

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770811592240                   -0.52    9.0         
  2   -2.772061582445       -2.90       -1.32    1.0    105ms
  3   -2.772083093485       -4.67       -2.45    1.0    124ms
  4   -2.772083339309       -6.61       -3.14    1.0    141ms
  5   -2.772083416538       -7.11       -3.97    2.0    123ms
  6   -2.772083417693       -8.94       -4.65    1.0    139ms
  7   -2.772083417810       -9.93       -5.40    2.0    127ms
  8   -2.772083417810      -12.58       -5.92    1.0    124ms
  9   -2.772083417811      -12.59       -6.84    2.0    139ms
 10   -2.772083417811   +  -14.88       -7.67    1.0    139ms
 11   -2.772083417811   +  -15.35       -8.32    1.0    118ms

Polarizability via ForwardDiff:       1.7725349655339555
Polarizability via finite difference: 1.7735577017380832
